In [1]:
import cv2
import numpy as np

In [2]:
def contourColor(colorName, mask, B, G, R,output):
  counter = 0
  contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  for pic, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if(area > 300):
      x, y, w, h = cv2.boundingRect(contour)
      colorImageFrame = cv2.rectangle(output, (x, y),(x + w, y + h),(B, G, R), 2)
      
      counter = counter + 1
  return counter

In [3]:


def detect_objects(image):
    
  """Detects objects in an image and draws contours around them.

  Args:
    image: A numpy array representing the image.

  Returns:
    A numpy array representing the image with the contours drawn on it.
  """

  # Blur the image to reduce noise and smooth out the edges.
  global flag
  blurred_image = cv2.blur(image, (3, 3))

  # Convert the image to grayscale.
  gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

  # Apply the Canny edge detector to find the edges of the objects in the image.
  canny_image = cv2.Canny(gray_image, 50, 190, 3)

  # Find the contours in the edge image.
  contours, _ = cv2.findContours(canny_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  flag=0
  for contour in contours:
    area=cv2.contourArea(contour)
    if area<100:
          continue
    approx=cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
    cv2.drawContours(image,[approx],0,(0,0,0),5)
    x= approx.ravel()[0]
    y= approx.ravel()[1]-10
    if len( approx)==3:
      flag=1
      cv2.putText(image,"Triangle",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,0,0))
    elif len(approx)==4:
      flag=1
      x, y, w, h = cv2.boundingRect(contour)
      aspectRatio = (float(w)/h)
      if aspectRatio >= 0.95 and aspectRatio <= 1.05:
        cv2.putText(image, "square", (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 0))
      else :
        cv2.putText(image, "rectangle", (x,y), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 0))  
    
    elif len( approx)==10:
      flag=1
      cv2.putText(image,"Star",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,0,0))
    elif len( approx)==8:
      flag=1
      cv2.putText(image,"Circle",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,0,0))

  return image

In [4]:
def preprocessing_image(image):
  
  output = cv2.resize(image, (600,600))
  hsvFrame = cv2.cvtColor(output, cv2.COLOR_RGB2HSV)
  return hsvFrame,output

In [5]:
def create_masks(hsvFrame):

  kernal = np.ones((5, 5), "uint8")
  
  red_lower = np.array([160, 170, 100], np.uint8)
  red_upper = np.array([180, 255, 255], np.uint8)
  red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)
  red_mask = cv2.dilate(red_mask, kernal)

  blue_lower = np.array([88, 80, 100], np.uint8)
  blue_upper = np.array([120, 255, 255], np.uint8)
  blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)
  blue_mask = cv2.dilate(blue_mask, kernal)

  green_lower = np.array([32, 70, 100], np.uint8)
  green_upper = np.array([60, 255, 155], np.uint8)
  green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)
  green_mask = cv2.dilate(green_mask, kernal)

  yellow_lower = np.array([27, 80, 180], np.uint8)
  yellow_upper = np.array([30, 255, 255], np.uint8)
  yellow_mask = cv2.inRange(hsvFrame, yellow_lower, yellow_upper)
  yellow_mask = cv2.dilate(yellow_mask, kernal)

  brown_lower = np.array([20, 100, 20], np.uint8)
  brown_upper = np.array([20, 255, 200], np.uint8) #15, 255, 150
  brown_mask = cv2.inRange(hsvFrame, brown_lower, brown_upper)
  brown_mask = cv2.dilate(brown_mask, kernal)

  orange_lower = np.array([5, 140, 170], np.uint8)
  orange_upper = np.array([13, 230, 255], np.uint8)
  orange_mask = cv2.inRange(hsvFrame, orange_lower, orange_upper)
  orange_mask = cv2.dilate(orange_mask, kernal)

  return red_mask,blue_mask,green_mask,yellow_mask,brown_mask,orange_mask

In [6]:
def counterText(x, y, color, colorCounter,output):
  cv2.putText(output, color + ": " + str(colorCounter), (x, y),cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,0,255), 1)

In [7]:
def detect_all_colors(red_mask,blue_mask,orange_mask,green_mask,yellow_mask,brown_mask,output):

  redCounter = contourColor('red',red_mask,0, 255, 0,output)
  counterText(50, 500, "RED", redCounter,output)
  blueCounter = contourColor('Blue', blue_mask, 0, 255, 0,output)
  counterText(150, 500, "BLUE", blueCounter,output)
  orangeCounter = contourColor('Orange', orange_mask, 0, 255, 0,output)
  counterText(280, 500, "Orange", orangeCounter,output)
  greenCounter = contourColor('Green', green_mask, 0, 255, 0,output)
  counterText(430, 500, "GREEN", greenCounter,output)
  yellowCounter = contourColor('Yellow', yellow_mask, 0, 255, 0,output)
  counterText(50,550, "YELLOW", yellowCounter,output)
  brownCounter = contourColor('Brown', brown_mask, 0, 255, 0,output)
  counterText(200,550, "BROWN", brownCounter,output)
  total_counter=redCounter+blueCounter+orangeCounter+greenCounter+yellowCounter+brownCounter
  counterText(50, 450, "total_count", total_counter,output)

In [8]:
cap=cv2.VideoCapture(0)
while True:
 ret,frame=cap.read()
 if not ret:
    break
 frame=cv2.resize(frame,(640,480))
 # Detect objects in the image and draw contours around them.
 image_with_contours = detect_objects(frame)

 hsvFrame,output=preprocessing_image(image_with_contours)
 red_mask,blue_mask,green_mask,yellow_mask,brown_mask,orange_mask=create_masks(hsvFrame)
 if flag==1:
     #print("g")
     detect_all_colors(red_mask,blue_mask,orange_mask,green_mask,yellow_mask,brown_mask,output)
 cv2.imshow("output",output)
 if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()